In [ ]:
import pandas as pd
import csv
from loader import *
from pair_finder import *
import time
from random import seed
import keyboard
from random import seed
from random import randint

In [ ]:
ratings_path_s = "DATA\\ratings.csv"
ratings_path = "DATA\\ratings.csv"
movies_path =  "DATA\\movies.csv"

MinScore = 4
MaxCombo = 4
MinFrequency = 0.1
ratings_stream = createRatingsStream(ratings_path_s, MinScore)
userBaskets, movie_list = CreateMovieBaskets(ratings_path, MinScore, 1000)
movies_df = ReadMovies(movies_path, movie_list)

In [ ]:
myApriori = myApriori(userBaskets, MinFrequency, MaxCombo)
c_combos = export_combos(myApriori)
s_combos = sampledApriori(100, ratings_stream, MinFrequency, MaxCombo)
print(get_scores(c_combos, s_combos))